# Bounds
The `Bounds` class is used to perform symbolic bounds checking for indices. There are two ways in which an index can be out of bounds: below the lower bound (underflow) or above the upper bound (overflow). Depending on what the situtation calls for, the bounds object can perform different actions.

Let's create a bounds object that does not contain any symbolic values and that ranges from 0 to 2, where `2` is non-inclusive (similar to how `range` behaves in Python).

In [1]:
from openfd.alpha import Bounds

In [2]:
B = Bounds(0, 2)

If an index is in bounds, we always get it back

In [3]:
B[1]

1

If we just need to check if an index is in bounds, then we can use the `is_in_bounds` method.

In [4]:
B.is_in_bounds(1)

True

In [5]:
B.is_in_bounds(-1)

False

Let us now turn our attention to symbolic bounds

In [6]:
from openfd.alpha import index
a, b = index.indices('a b')


In [7]:
B = Bounds(a, b)

As before, as long as the index is in bounds, we get it back

In [8]:
B[a]

a

In [9]:
B[a+1]

a + 1

In [10]:
B[b-1]

b - 1

Again, we can check if it is in bounds

In [11]:
B.is_in_bounds(a)

True

In [12]:
B.is_in_bounds(a-1)

False

Some care has to be taken when checking symbolic bounds. The bounds checker is somewhat primitive. Although, we know that `a+b` should be out of bounds, we have not yet added support to check for complicated expressions of this nature. If it is not possible to determine if index expression is in bounds or not, an exception is raised.

In [13]:
# B.is_in_bounds(a+b) # Uncomment to raise an IndexException

## Handling indices that are out of bounds

So far, all of our index expressions have been in bounds. What happens if an index is out of bounds? The answer depends on if the index falls below or above the acceptable range set by a bounds object.

Lower out of bounds errors causes a wrap-around, and this behavior is similar to how `list` behaves in Python

In [14]:
B[a-1]

b - 1

On the other hand, an `OutOfBoundsException` is raised if the upper bounds is exceeded

In [15]:
# B[b] # Uncommment to raise UpperOutOfBoundsException

It is possible to override the default action to take on either a lower or upper out of bounds error.

In [16]:
B = Bounds(a, b, lower_out_of_bounds='wrap-around', upper_out_of_bounds='wrap-around')

Now, no exception is raised when the upper bound is exceeded. Instead wrap-around takes place.

In [17]:
B[b]

a

There are three options to choose from: `'wrap-around'`, `'raise exception'`, and `'no action'`. We have seen examples of the first two options. The last option, as the name implies, does nothing when an index is out of bounds. 

In [18]:
B = Bounds(a, b, lower_out_of_bounds='no action', upper_out_of_bounds='no action')

In [19]:
B[a-1]

a - 1

In [20]:
B[b]

b

The final feature to discuss is how one goes about implementing a custom action. There are at least two solutions to accomplish this task. One way is to build a new class that inherits from `Bounds` and overrides the `lower_out_of_bounds_action` and `upper_out_of_bounds_action` method. A second solution is to catch the exceptions raised when an out of bounds error occurs. What solution to use depends on the situation. 

### Overriding the out_of_bounds_action methods
Let's give an example of how to always return `0` when either a lower or upper bounds error occurs using method overriding. While it is possible to solve the problem as demonstrated below, it has a caveat. The problem is that the `lower_out_of_bounds` and `upper_out_of_bounds` attributes become meaningless and therefore make the code more difficult to read. 

In [21]:
class ExampleBounds(Bounds):
    def lower_out_of_bounds_action(self, idx, *args, **kw_args):
        return 0
    def upper_out_of_bounds_action(self, idx, *args, **kw_args):
        return 0

After constructing this class, we can now build a new object that achieves the desired behavior.

In [22]:
example = ExampleBounds(a, b)
example[a-1]

0

In [23]:
example[b+1]

0

### Exception handling
The second solution involves catching the out of bounds exceptions and implement what action to take when they are raised. This solution does not make the attributes `lower_out_of_bounds` and `upper_out_of_bounds` meaningless and also makes it possible to pass additional arguments, if needed.

In [24]:
from openfd.alpha.bounds import LowerOutOfBoundsException, UpperOutOfBoundsException

First, we need to make sure that an exception is raised when either a lower or upper out of bounds error occurs.

In [25]:
example = Bounds(a, b, lower_out_of_bounds='raise exception', upper_out_of_bounds='raise exception')


Then we can go ahead and catch and handle the exceptions.

In [26]:
def get_item(bounds, index, *arg, **kw_args):
    try:
        return bounds[index]
    except LowerOutOfBoundsException:
        return 0
    except UpperOutOfBoundsException:
        return 0

In [27]:
get_item(example, a-1)

0

In [28]:
get_item(example, b)

0